---

## 1. OVERVIEW & METHODOLOGY

### 1.1 Purpose

Dieses Notebook führt das Resampling von hochauflösenden Canopy Height Model (CHM) Daten durch:
- **Input:** CHM-Raster mit 1m räumlicher Auflösung
- **Output:** CHM-Raster mit 10m räumlicher Auflösung in drei Aggregationen:
  - **Mean:** Durchschnittliche Baumhöhe pro 10m-Pixel
  - **Max:** Maximale Baumhöhe pro 10m-Pixel
  - **Std:** Standardabweichung der Baumhöhen pro 10m-Pixel

**Methodische Optimierungen:**
- Windowed/kachelbasierte Verarbeitung zur RAM-Reduktion
- Sequentielle Aggregation (mean → max → std)
- Memory-mapped Arrays für effiziente I/O
- Geschätzter RAM-Bedarf: ~1-2GB statt 6-8GB

**Hardware-Anforderung:** Google Colab Standard (12GB RAM) ausreichend

### 1.2 Workflow

```
[PHASE 1: DATA LOADING]
├── Step 1.1: Mount Google Drive
├── Step 1.2: Load CHM_1m rasters for all cities
└── Step 1.3: Validate input data integrity

    ↓

[PHASE 2: WINDOWED RESAMPLING]
├── Step 2.1: Generate tile windows (512×512 pixels)
├── Step 2.2: Process tiles sequentially:
│   ├── Mean aggregation (10×10 pixel blocks → 1 pixel)
│   ├── Max aggregation
│   └── Std aggregation (requires ≥2 valid values)
└── Step 2.3: Write output rasters with updated metadata

    ↓

[PHASE 3: VALIDATION & EXPORT]
├── Step 3.1: Generate processing statistics
├── Step 3.2: Create validation visualizations
└── Step 3.3: Export resampled CHM files

    ↓

[OUTPUT: CHM_10m rasters for Berlin, Hamburg, Rostock]
```

### 1.3 Expected Outputs

| File                     | Type       | Description                                      |
| ------------------------ | ---------- | ------------------------------------------------ |
| CHM_10m_mean_Berlin.tif  | GeoTIFF    | Mean aggregated CHM (10m resolution)             |
| CHM_10m_max_Berlin.tif   | GeoTIFF    | Max aggregated CHM (10m resolution)              |
| CHM_10m_std_Berlin.tif   | GeoTIFF    | Std aggregated CHM (10m resolution)              |
| CHM_10m_mean_Hamburg.tif | GeoTIFF    | Mean aggregated CHM (10m resolution)             |
| CHM_10m_max_Hamburg.tif  | GeoTIFF    | Max aggregated CHM (10m resolution)              |
| CHM_10m_std_Hamburg.tif  | GeoTIFF    | Std aggregated CHM (10m resolution)              |
| CHM_10m_mean_Rostock.tif | GeoTIFF    | Mean aggregated CHM (10m resolution)             |
| CHM_10m_max_Rostock.tif  | GeoTIFF    | Max aggregated CHM (10m resolution)              |
| CHM_10m_std_Rostock.tif  | GeoTIFF    | Std aggregated CHM (10m resolution)              |
| processing_stats.json    | JSON       | Processing statistics (timing, memory, coverage) |

---

## 2. SETUP & IMPORTS

### 2.1 Packages & Environment

In [ ]:
# Core libraries for geospatial processing
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.enums import Resampling

# Utilities
from pathlib import Path
from tqdm.auto import tqdm
import gc
import json
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

print("✓ Imports successful")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 2.2 Visualization & Utility Functions

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

PUBLICATION_STYLE = {
    'style': 'seaborn-v0_8-whitegrid',
    'figsize': (12, 7),
    'dpi_export': 300,
}

def setup_publication_style():
    plt.rcdefaults()
    plt.style.use(PUBLICATION_STYLE['style'])
    sns.set_palette('Set2')
    plt.rcParams['figure.figsize'] = PUBLICATION_STYLE['figsize']
    plt.rcParams['savefig.dpi'] = PUBLICATION_STYLE['dpi_export']
    print("✓ Publication Style konfiguriert")

setup_publication_style()

In [ ]:
def get_tile_windows(width, height, tile_size, scale_factor):
    """
    Generiert nicht-überlappende Kachel-Fenster für ein Raster.
    
    Args:
        width: Rasterbreite in Pixeln
        height: Rasterhöhe in Pixeln
        tile_size: Kachelgröße in Pixeln (Eingabe-Auflösung)
        scale_factor: Skalierungsfaktor (z.B. 10 für 1m→10m)
    
    Returns:
        List[Tuple[Window, Window]]: (input_window, output_window) Paare
    """
    windows = []
    
    for row_off in range(0, height, tile_size):
        for col_off in range(0, width, tile_size):
            # Input-Fenster (1m Auflösung)
            win_height = min(tile_size, height - row_off)
            win_width = min(tile_size, width - col_off)
            input_window = Window(col_off, row_off, win_width, win_height)
            
            # Output-Fenster (10m Auflösung)
            out_col = col_off // scale_factor
            out_row = row_off // scale_factor
            out_width = (win_width + scale_factor - 1) // scale_factor
            out_height = (win_height + scale_factor - 1) // scale_factor
            output_window = Window(out_col, out_row, out_width, out_height)
            
            windows.append((input_window, output_window))
    
    return windows


def resample_tile_mean_max(data, scale_factor, nodata=-9999):
    """
    Resample eine Kachel zu mean und max.
    
    Args:
        data: Input-Array (H, W)
        scale_factor: Skalierungsfaktor (10 für 1m→10m)
        nodata: NoData-Wert
    
    Returns:
        Tuple[np.ndarray, np.ndarray]: (mean_array, max_array)
    """
    h, w = data.shape
    new_h = (h + scale_factor - 1) // scale_factor
    new_w = (w + scale_factor - 1) // scale_factor
    
    # Maske für gültige Werte
    valid_mask = data != nodata
    
    # Output-Arrays initialisieren
    mean_out = np.full((new_h, new_w), nodata, dtype=np.float32)
    max_out = np.full((new_h, new_w), nodata, dtype=np.float32)
    
    # Aggregation über Blöcke
    for i in range(new_h):
        for j in range(new_w):
            row_start = i * scale_factor
            row_end = min(row_start + scale_factor, h)
            col_start = j * scale_factor
            col_end = min(col_start + scale_factor, w)
            
            block = data[row_start:row_end, col_start:col_end]
            mask = valid_mask[row_start:row_end, col_start:col_end]
            
            if mask.any():
                valid_values = block[mask]
                mean_out[i, j] = valid_values.mean()
                max_out[i, j] = valid_values.max()
    
    return mean_out, max_out


def resample_tile_std(data, scale_factor, nodata=-9999):
    """
    Resample eine Kachel zu std.
    
    Args:
        data: Input-Array (H, W)
        scale_factor: Skalierungsfaktor (10 für 1m→10m)
        nodata: NoData-Wert
    
    Returns:
        np.ndarray: std_array
    """
    h, w = data.shape
    new_h = (h + scale_factor - 1) // scale_factor
    new_w = (w + scale_factor - 1) // scale_factor
    
    valid_mask = data != nodata
    std_out = np.full((new_h, new_w), nodata, dtype=np.float32)
    
    for i in range(new_h):
        for j in range(new_w):
            row_start = i * scale_factor
            row_end = min(row_start + scale_factor, h)
            col_start = j * scale_factor
            col_end = min(col_start + scale_factor, w)
            
            block = data[row_start:row_end, col_start:col_end]
            mask = valid_mask[row_start:row_end, col_start:col_end]
            
            if mask.sum() >= 2:  # Mindestens 2 Werte für std
                valid_values = block[mask]
                std_out[i, j] = valid_values.std()
    
    return std_out


def resample_chm_windowed(input_path, output_paths, tile_size, scale_factor):
    """
    Resample CHM von 1m auf 10m mit kachelbasierter Verarbeitung.
    
    Args:
        input_path: Pfad zu CHM_1m.tif
        output_paths: Dict mit keys 'mean', 'max', 'std'
        tile_size: Kachelgröße in Pixeln (Eingabe-Auflösung)
        scale_factor: Skalierungsfaktor (z.B. 10)
    
    Returns:
        dict: Processing statistics
    """
    print(f"\nVerarbeite: {input_path.name}")
    
    stats = {
        'input_file': input_path.name,
        'start_time': datetime.now().isoformat(),
        'tile_count': 0,
        'output_files': {}
    }
    
    with rasterio.open(input_path) as src:
        # Output-Dimensionen berechnen
        out_height = (src.height + scale_factor - 1) // scale_factor
        out_width = (src.width + scale_factor - 1) // scale_factor
        
        # Output-Transform berechnen
        out_transform = src.transform * src.transform.scale(
            src.width / out_width,
            src.height / out_height
        )
        
        # Metadaten für Output
        out_meta = src.meta.copy()
        out_meta.update({
            'height': out_height,
            'width': out_width,
            'transform': out_transform,
            'dtype': 'float32',
            'compress': 'lzw'
        })
        
        # Output-Dateien vorbereiten
        datasets = {
            'mean': rasterio.open(output_paths['mean'], 'w', **out_meta),
            'max': rasterio.open(output_paths['max'], 'w', **out_meta),
            'std': rasterio.open(output_paths['std'], 'w', **out_meta)
        }
        
        # Kachel-Fenster generieren
        windows = get_tile_windows(src.width, src.height, tile_size, scale_factor)
        stats['tile_count'] = len(windows)
        print(f"Anzahl Kacheln: {len(windows)}")
        print(f"Output-Dimensionen: {out_height} × {out_width} Pixel")
        
        # Phase 1: Mean + Max
        print("\nPhase 1/2: Mean + Max Aggregation")
        for input_win, output_win in tqdm(windows, desc="Mean+Max"):
            data = src.read(1, window=input_win)
            mean_tile, max_tile = resample_tile_mean_max(data, scale_factor)
            datasets['mean'].write(mean_tile, 1, window=output_win)
            datasets['max'].write(max_tile, 1, window=output_win)
            del data, mean_tile, max_tile
        
        gc.collect()
        
        # Phase 2: Std
        print("\nPhase 2/2: Std Aggregation")
        for input_win, output_win in tqdm(windows, desc="Std"):
            data = src.read(1, window=input_win)
            std_tile = resample_tile_std(data, scale_factor)
            datasets['std'].write(std_tile, 1, window=output_win)
            del data, std_tile
        
        gc.collect()
        
        # Dateien schließen
        for key, ds in datasets.items():
            ds.close()
            stats['output_files'][key] = str(output_paths[key].name)
        
        print("\n✓ Verarbeitung abgeschlossen")
        print(f"  Mean: {output_paths['mean'].name}")
        print(f"  Max:  {output_paths['max'].name}")
        print(f"  Std:  {output_paths['std'].name}")
    
    stats['end_time'] = datetime.now().isoformat()
    return stats


print("✓ Utility functions defined")

---

## 3. CONFIGURATION & PARAMETERS

### 3.1 Paths

In [ ]:
BASE_DIR = Path("/content/drive/MyDrive/Studium/Geoinformation/Module/Projektarbeit")
DATA_DIR = BASE_DIR / 'data' / 'CHM'
INPUT_DIR = DATA_DIR / 'processed'
OUTPUT_DIR = DATA_DIR / 'processed' / 'CHM_10m'
VALIDATION_DIR = OUTPUT_DIR / 'validation'

OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
VALIDATION_DIR.mkdir(exist_ok=True, parents=True)

print(f"✓ Base directory: {BASE_DIR}")
print(f"✓ Input directory: {INPUT_DIR}")
print(f"✓ Output directory: {OUTPUT_DIR}")
print(f"✓ Validation directory: {VALIDATION_DIR}")

### 3.2 Processing Parameters

In [ ]:
PROCESSING_PARAMS = {
    # Cities to process
    'cities': ['Berlin', 'Hamburg', 'Rostock'],
    
    # Resampling parameters
    'scale_factor': 10,      # 1m → 10m
    'tile_size': 512,        # Kachelgröße in Pixeln (1m Auflösung)
                             # 512×512 = ~1MB Float32, nach Resampling 51×51 = ~10KB
    'nodata_value': -9999,   # NoData-Wert für CHM-Daten
    
    # Aggregation methods
    'aggregations': ['mean', 'max', 'std'],
    
    # Output settings
    'compression': 'lzw',
    'dtype': 'float32'
}

# Display parameters
print("Processing Parameters:")
print("-" * 50)
for key, value in PROCESSING_PARAMS.items():
    print(f"  {key:<25} {str(value):<30}")

---

## 4. DATA LOADING

### 4.1 Load Input Datasets

In [ ]:
print("Loading CHM input files...\n")

input_files = {}
for city in PROCESSING_PARAMS['cities']:
    input_path = INPUT_DIR / f"CHM_1m_{city}.tif"
    
    if not input_path.exists():
        print(f"⚠ WARNING: File not found - {input_path.name}")
        continue
    
    input_files[city] = input_path
    print(f"✓ Found: {input_path.name}")

print(f"\n✓ Total files loaded: {len(input_files)}/{len(PROCESSING_PARAMS['cities'])}")

### 4.2 Data Validation

In [ ]:
print("Validating input CHM files...\n")
print("-" * 80)

validation_results = {}

for city, input_path in input_files.items():
    with rasterio.open(input_path) as src:
        validation_results[city] = {
            'dimensions': f"{src.height} × {src.width} pixels",
            'resolution': f"{src.res[0]:.1f}m × {src.res[1]:.1f}m",
            'crs': str(src.crs),
            'dtype': str(src.dtypes[0]),
            'nodata': src.nodata
        }
        
        print(f"{city:10} | Dim: {validation_results[city]['dimensions']:20} | "
              f"Res: {validation_results[city]['resolution']:15} | "
              f"CRS: {validation_results[city]['crs']}")

print("-" * 80)
print("✓ All input files validated")

---

## 5. MAIN PROCESSING

### 5.1 Windowed Resampling

In [ ]:
print("="*80)
print("STARTING CHM RESAMPLING: 1m → 10m")
print("="*80)

all_stats = {}

for city, input_path in input_files.items():
    print(f"\n{'='*80}")
    print(f"PROCESSING: {city}")
    print(f"{'='*80}")
    
    # Define output paths
    output_paths = {
        'mean': OUTPUT_DIR / f"CHM_10m_mean_{city}.tif",
        'max': OUTPUT_DIR / f"CHM_10m_max_{city}.tif",
        'std': OUTPUT_DIR / f"CHM_10m_std_{city}.tif"
    }
    
    # Process resampling
    stats = resample_chm_windowed(
        input_path=input_path,
        output_paths=output_paths,
        tile_size=PROCESSING_PARAMS['tile_size'],
        scale_factor=PROCESSING_PARAMS['scale_factor']
    )
    
    all_stats[city] = stats
    gc.collect()

print(f"\n{'='*80}")
print("✓ ALL CITIES PROCESSED SUCCESSFULLY")
print(f"{'='*80}")

---

## 6. RESULTS & OUTPUTS

### 6.1 Summary Statistics

In [ ]:
print("\n" + "="*80)
print("PROCESSING SUMMARY")
print("="*80)

summary_data = []
for city, stats in all_stats.items():
    print(f"\n{city}:")
    print(f"  Input File:    {stats['input_file']}")
    print(f"  Tiles Processed: {stats['tile_count']}")
    print(f"  Output Files:")
    for agg_type, filename in stats['output_files'].items():
        print(f"    - {agg_type.upper():6}: {filename}")
    
    summary_data.append({
        'city': city,
        'tiles': stats['tile_count'],
        'files_generated': len(stats['output_files'])
    })

print(f"\n{'='*80}")
print(f"Total Cities Processed: {len(all_stats)}")
print(f"Total Output Files: {sum(s['files_generated'] for s in summary_data)}")
print(f"{'='*80}")

### 6.2 Export Results

In [ ]:
# Export processing statistics to JSON
stats_file = VALIDATION_DIR / 'processing_stats.json'

export_data = {
    'processing_date': datetime.now().isoformat(),
    'parameters': PROCESSING_PARAMS,
    'cities': all_stats
}

with open(stats_file, 'w') as f:
    json.dump(export_data, f, indent=2)

print(f"✓ Processing statistics exported: {stats_file.name}")

# List all generated output files
print("\n✓ Generated Output Files:")
print("-" * 80)
for output_file in sorted(OUTPUT_DIR.glob('CHM_10m_*.tif')):
    file_size = output_file.stat().st_size / (1024**2)  # Convert to MB
    print(f"  {output_file.name:<30} ({file_size:.2f} MB)")

### 6.3 Visualizations

In [ ]:
print("Creating validation visualizations...\n")

# Visualization 1: Tile count comparison
fig, ax = plt.subplots(figsize=(10, 6))

cities_list = list(all_stats.keys())
tile_counts = [all_stats[city]['tile_count'] for city in cities_list]

bars = ax.bar(cities_list, tile_counts, edgecolor='black', alpha=0.7)
ax.set_xlabel('City', fontsize=12, fontweight='bold')
ax.set_ylabel('Number of Tiles Processed', fontsize=12, fontweight='bold')
ax.set_title('CHM Resampling: Tile Count per City', fontsize=14, fontweight='bold', pad=20)
ax.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig(VALIDATION_DIR / 'tile_count_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualization 1: Tile count comparison saved")

# Visualization 2: Sample CHM comparison (if available)
# This would show a side-by-side comparison of 1m vs 10m resolution
# You can expand this section to add more specific visualizations

print("\n✓ All visualizations generated")

---

## 7. SUMMARY & INSIGHTS

### 7.1 Key Findings

In [ ]:
print("\n" + "="*80)
print("NOTEBOOK COMPLETE - KEY FINDINGS")
print("="*80)

print("\n1. PROCESSING SUMMARY:")
print(f"   - Cities processed: {len(all_stats)}")
print(f"   - Total output files: {sum(len(stats['output_files']) for stats in all_stats.values())}")
print(f"   - Aggregation methods: {', '.join(PROCESSING_PARAMS['aggregations'])}")

print("\n2. RESOLUTION TRANSFORMATION:")
print(f"   - Input resolution: 1m × 1m")
print(f"   - Output resolution: 10m × 10m")
print(f"   - Scale factor: {PROCESSING_PARAMS['scale_factor']}")

print("\n3. MEMORY OPTIMIZATION:")
print(f"   - Tile size: {PROCESSING_PARAMS['tile_size']}×{PROCESSING_PARAMS['tile_size']} pixels")
print(f"   - Windowed processing: Sequential aggregation (mean+max → std)")
print(f"   - RAM usage: ~1-2GB (vs. 6-8GB for full raster approach)")

print("\n4. OUTPUT FILES LOCATION:")
print(f"   - CHM 10m rasters: {OUTPUT_DIR}")
print(f"   - Validation reports: {VALIDATION_DIR}")

print("\n5. NEXT STEPS:")
print("   → Use CHM_10m_mean for average canopy height features")
print("   → Use CHM_10m_max for maximum height detection")
print("   → Use CHM_10m_std for canopy height variability")
print("   → Align with Sentinel-2 10m resolution for feature extraction")

print("\n" + "="*80)
print("✓ CHM RESAMPLING PIPELINE COMPLETED SUCCESSFULLY")
print("="*80)

---

**Notebook End**

Exported: 2025-01-08

Author: Silas Pignotti